In [1]:
!pip install nltk

In [3]:
!pip install textblob

   ---------------------------------------- 0.0/624.3 kB ? eta -:--:--
   ---------------------------------------- 624.3/624.3 kB 3.9 MB/s eta 0:00:00


In [101]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from datetime import datetime

In [103]:
df = pd.read_csv('C:\\Users\\Admin\\excel\\top10K-TMDB-movies.csv')

In [105]:
df.describe()

,id,popularity,vote_average,vote_count
count,10000.000000,10000.000000,10000.000000,10000.000000
mean,161243.505000,34.697267,6.621150,1547.309400
std,211422.046043,211.684175,0.766231,2648.295789
min,5.000000,0.600000,4.600000,200.000000
25%,10127.750000,9.154750,6.100000,315.000000
50%,30002.500000,13.637500,6.600000,583.500000
75%,310133.500000,25.651250,7.200000,1460.000000
max,934761.000000,10436.917000,8.700000,31917.000000


In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 10000 non-null  int64  
 1   title              10000 non-null  object 
 2   genre              9997 non-null   object 
 3   original_language  10000 non-null  object 
 4   overview           9987 non-null   object 
 5   popularity         10000 non-null  float64
 6   release_date       10000 non-null  object 
 7   vote_average       10000 non-null  float64
 8   vote_count         10000 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 703.3+ KB


In [111]:
df.columns

Index(['id', 'title', 'genre', 'original_language', 'overview', 'popularity',
       'release_date', 'vote_average', 'vote_count'],
      dtype='object')

In [113]:
df.isna().sum()

id                    0
title                 0
genre                 3
original_language     0
overview             13
popularity            0
release_date          0
vote_average          0
vote_count            0
dtype: int64

In [115]:
df['overview'].fillna('No overview available', inplace=True)
df['genre'].fillna('Unknown genre', inplace=True)

In [117]:
df.isna().sum()

id                   0
title                0
genre                0
original_language    0
overview             0
popularity           0
release_date         0
vote_average         0
vote_count           0
dtype: int64

In [121]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [123]:
import nltk
print(nltk.data.path)


['C:\\Users\\Admin/nltk_data', 'C:\\Users\\Admin\\anaconda3\\nltk_data', 'C:\\Users\\Admin\\anaconda3\\share\\nltk_data', 'C:\\Users\\Admin\\anaconda3\\lib\\nltk_data', 'C:\\Users\\Admin\\AppData\\Roaming\\nltk_data', 'C:\\nltk_data', 'D:\\nltk_data', 'E:\\nltk_data']


In [127]:
nltk.data.path.append('C:/Users/Admin/nltk_data')

In [131]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\abc.zip.
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers\averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       tagge

True

In [133]:
df['overview'] = df['overview'].str.lower()
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    tokens = word_tokenize(text)
    filtered_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words and word.isalnum()]
    return ' '.join(filtered_tokens)

df['overview'] = df['overview'].apply(preprocess_text)

In [153]:
encoder = OneHotEncoder()
genre_encoded = encoder.fit_transform(df[['genre']]).toarray()
language_encoded = encoder.fit_transform(df[['original_language']]).toarray()

In [159]:
# Import necessary library
from sklearn.preprocessing import OneHotEncoder

# Perform one-hot encoding for 'genre'
encoder = OneHotEncoder(sparse_output=False)
genre_encoded = encoder.fit_transform(df[['genre']])  # Perform encoding

# Correctly generate column names
genre_df = pd.DataFrame(genre_encoded, columns=encoder.get_feature_names_out(['genre']))

# Similarly, for 'original_language'
language_encoded = encoder.fit_transform(df[['original_language']])
language_df = pd.DataFrame(language_encoded, columns=encoder.get_feature_names_out(['original_language']))

# Concatenate the encoded DataFrames back to the main DataFrame
df = pd.concat([df, genre_df, language_df], axis=1)

# Check the final shape of the DataFrame
print(df.shape)

(10000, 2176)


In [161]:
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
df['release_year'] = df['release_date'].dt.year

In [163]:
scaler = MinMaxScaler()
numerical_columns = ['popularity', 'vote_average', 'vote_count']
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

In [167]:
df.head()

,title,genre,original_language,overview,popularity,release_date,vote_average,vote_count,release_year,genre_Action,...,original_language_ru,original_language_sh,original_language_sr,original_language_sv,original_language_te,original_language_th,original_language_tr,original_language_uk,original_language_xx,original_language_zh
0,The Shawshank Redemption,"Drama,Crime",en,framed 1940s double murder wife lover upstandi...,0.008957,1994-09-23,1.00000,0.682978,1994,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Dilwale Dulhania Le Jayenge,"Comedy,Drama,Romance",hi,raj rich carefree second generation nri simran...,0.002377,1995-10-19,1.00000,0.111328,1995,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,The Godfather,"Drama,Crime",en,spanning year 1945 1955 chronicle fictional co...,0.008622,1972-03-14,1.00000,0.506984,1972,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Schindler's List,"Drama,History,War",en,true story businessman oskar schindler saved t...,0.004231,1993-12-15,0.97561,0.402276,1993,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,The Godfather: Part II,"Drama,Crime",en,continuing saga corleone crime family young vi...,0.005476,1974-12-20,0.97561,0.303024,1974,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
